In [6]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [19]:
pool_train = pd.read_csv(
    "hpdatastrip.csv",
    names=["last_changed","hdpower","poolheatedtemp","poolreturntemp","dark_sky_temperture","efficiency"
])

pool_train.head()

,last_changed,hdpower,poolheatedtemp,poolreturntemp,dark_sky_temperture,efficiency
0,2020-06-22 16:30:10.468728-04:00,4742,94,90,90.4,8.435259
1,2020-06-22 16:30:40.556357-04:00,4742,93,90,90.4,6.326445
2,2020-06-23 13:28:57.028470-04:00,4461,88,87,84.7,2.241650
3,2020-06-23 13:28:57.047076-04:00,4217,88,87,84.7,2.371354
4,2020-06-23 13:28:59.058786-04:00,4275,88,87,84.7,2.339181


In [47]:
pool_features = pool_train.copy()
pool_labels = pool_features.pop("efficiency")
pool_extran1 = pool_features.pop("last_changed")
pool_entran2 = pool_features.pop("hdpower")
pool_features_ary = np.array(pool_features)
pool_features_ary

array([[ 94. ,  90. ,  90.4],
       [ 93. ,  90. ,  90.4],
       [ 88. ,  87. ,  84.7],
       ...,
       [101. ,  95. ,  88. ],
       [101. ,  95. ,  88. ],
       [101. ,  95. ,  88. ]])

In [28]:
pool_model = tf.keras.Sequential([
  layers.Dense(64),
  layers.Dense(1)
])

pool_model.compile(loss = tf.losses.MeanSquaredError(),
                      optimizer = tf.optimizers.Adam())

In [48]:
pool_model.fit(pool_features_ary, pool_labels, epochs=10)


Epoch 1/10
2614/2614 [==============================] - 5s 2ms/step - loss: 0.2797
Epoch 2/10
2614/2614 [==============================] - 5s 2ms/step - loss: 0.2715
Epoch 3/10
2614/2614 [==============================] - 6s 2ms/step - loss: 0.2812
Epoch 4/10
2614/2614 [==============================] - 8s 3ms/step - loss: 0.2592
Epoch 5/10
2614/2614 [==============================] - 5s 2ms/step - loss: 0.2719
Epoch 6/10
2614/2614 [==============================] - 5s 2ms/step - loss: 0.2598
Epoch 7/10
2614/2614 [==============================] - 4s 2ms/step - loss: 0.2602
Epoch 8/10
2614/2614 [==============================] - 8s 3ms/step - loss: 0.2617
Epoch 9/10
2614/2614 [==============================] - 5s 2ms/step - loss: 0.2523
Epoch 10/10
2614/2614 [==============================] - 6s 2ms/step - loss: 0.2553


In [50]:
normalize = preprocessing.Normalization()


In [51]:
normalize.adapt(pool_features_ary)


In [52]:
norm_pool_model = tf.keras.Sequential([
  normalize,
  layers.Dense(64),
  layers.Dense(1)
])

norm_pool_model.compile(loss = tf.losses.MeanSquaredError(),
                           optimizer = tf.optimizers.Adam())

norm_pool_model.fit(pool_features_ary, pool_labels, epochs=10)

Epoch 1/10
2614/2614 [==============================] - 5s 2ms/step - loss: 3.4672
Epoch 2/10
2614/2614 [==============================] - 5s 2ms/step - loss: 0.1951
Epoch 3/10
2614/2614 [==============================] - 7s 3ms/step - loss: 0.1953
Epoch 4/10
2614/2614 [==============================] - 6s 2ms/step - loss: 0.1958
Epoch 5/10
2614/2614 [==============================] - 7s 3ms/step - loss: 0.1954
Epoch 6/10
2614/2614 [==============================] - 8s 3ms/step - loss: 0.1952
Epoch 7/10
2614/2614 [==============================] - 7s 3ms/step - loss: 0.1938
Epoch 8/10
2614/2614 [==============================] - 5s 2ms/step - loss: 0.1954
Epoch 9/10
2614/2614 [==============================] - 8s 3ms/step - loss: 0.1935
Epoch 10/10
2614/2614 [==============================] - 5s 2ms/step - loss: 0.1949


In [61]:
dark_sky_temperture = np.array(pool_features['dark_sky_temperture'])
dark_sky_temperture[:10]

array([90.4, 90.4, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7, 84.7])

In [62]:
poolheatedtemp = np.array(pool_features['poolheatedtemp'])
poolheatedtemp[:10]

array([94, 93, 88, 88, 88, 88, 88, 88, 88, 88])

In [63]:
poolreturntemp = np.array(pool_features['poolheatedtemp'])
poolreturntemp[:10]

array([94, 93, 88, 88, 88, 88, 88, 88, 88, 88])

In [117]:
dp = np.array([[94, 90,90.4]])
print(dp)

eff = pool_model.predict(dp)
print(eff)


[[94.  90.  90.4]]
[[8.484]]


In [120]:
dp = np.array([[93, 90,90.4]])
print(dp)

eff = pool_model.predict(dp)
print(eff)

[[93.  90.  90.4]]
[[6.456]]


In [119]:
dp = np.array([[101, 95, 88]])
print(dp)

eff = pool_model.predict(dp)
print(eff)

[[101  95  88]]
[[12.373]]


In [37]:
pool_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 64)                256       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________
